In [1]:
import os
import torch as T
import torch.nn.Functional as F
import numpy as np
import torch.nn as nn
from torch.distributions.normal import Normal
import torch.optim as optim
from torch.distributions.normal import Normal
import numpy as np

OSError: [WinError 127] The specified procedure could not be found. Error loading "C:\Users\akafi\anaconda3\envs\aima\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.

In [ ]:
class replayBuffer:
    def __init__(self, max_size, input_shape, n_actions):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.state_memory = np.zeros((self.mem_size, *input_shape))
        self.new_state_memory = np.zeros((self.mem_size, *input_shape))
        self.action_memory = np.zeros((self.mem_size, n_actions))
        self.reward_memory = np.zeros(self.mem_size)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.bool)
    
    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        
        self.state_memory[index] = state
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.new_state_memory[index] = state_
        self.terminal_memory[index] = done
        
        self.mem_cntr += 1
        
    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        
        batch = np.random.choice(max_mem, batch_size)
        
        states = self.state_memory[batch]
        states_ = self.new_state_memory[batch]
        actions = self.action_memory[batch]
        reward = self.reward_memory[batch]
        dones = self.terminal_memory[batch]
        
        return states_, actions, states, rewards, dones

In [ ]:
# Critic Network
class CriticNetwork(nn.Module):
    def __init__(self, beta, input_dims, fc1_dims=256, fc2_dims=256, n_actions, name='critic', chkpt_dir='tmp/sac'):
        super(CriticNetwork, self).__init__()
        
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.name = name
        self.checkpoint_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.checkpoint_dir, name + '_sac')
        
        self.fc1 = nn.Linear(self.input_dims[0] + n_actions, self.fc1)
        self.fc2 = nn.Linear(self.self.fc1_dims, self.fc2_dims)
        self.q = nn.Linear(self.fc2_dims, 1)
        
        self.optimizer = optim.Adam(self.parameters(), lr=beta)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        
        self.to(self.device)
    
    def forward(self, state, actions):
        action_value = self.fc1(T.cat([state, action], dim = 1))
        action_value = F.relu(actions_values)
        action_value = self.fc2(action_value)
        action_value = F.relu(action_value)
        
        q = self.q(action_value)
        
        return q
    
    def save_checkpoint(self):
        T.save(self.state_dict(), self.checkpoint_file)
        
    def load_checkpoint(self):
        self.load_state_dict_dict(T.load(self.checkpoint_file))
        
        

In [ ]:
# Value Network
class ValueNetwork(nn.Module):
    def __init__(self, beta, input_dims, fc1_dims=256, fc2_dims=256, name='value', chkpt_dir='tmp/sac'):
        super(ValueNetwork, self).__init__()
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.name = name
        self.checkpoint_dir = chkpt_dir 
        self.checkpoint_files = os.path.join(self.checkpoint_dir, name +'_sac')
        
        self.fc1 = nn.Linear(*self.input_dim, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.v = nn.Linear(self.fc2_dims, 1)
        
        self.optimizer = optim.Adam(self.parameters(), lr=beta)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        
        self.to(self.device)
        
    def forward(self, state):
        state_value = self.fc1(state)
        state_value = F.relu(state_value)
        state_value = self.fc2(state_value)
        state_value = F.relu(state_value)
        
        q = self.q(state_value)
        
        return q
    
    def save_checkpoint(self):
        T.save(self.state_dict(), self.checkpoint_file)
        
    def load_checkpoint(self):
        self.load_state_dict_dict(T.load(self.checkpoint_file))
        

In [9]:
# Actor Network
class ActorNetwork(nn.Module):
    def __init__(self, alpha, input_dims,  max_action, n_actions, fc1_dims=256, fc2_dims=256,  name='actor', chkpt_dir='tmp/sac'):
        super(ActorNetwork, self).__init()
        
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.name = name
        self.checkpoint_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.checkpoint_dir, name+'_sac')
        self.max_action = max_action
        self.reparam_noise = 1e-6
        
        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.mu = nn.Linear(self.fc2_dims, self.n_actions)
        self.sigma = nn.Linear(self.fc2_dims, self.n_actions)
        
        self.optimizers = optim.Adam(self.parameters(), lr=alpha)
        self.device() = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        
        self.to(self.device)
        
    def forward(self,state):
        prob = self.fc1(state)
        prob = F.relu(prob)
        prob = self.fc2(prob)
        prob = F.relu(prob)
        
        mu = self.mu(prob)
        sigma = self.sigma(prob)
        
        sigma = T.clamp(sigma, min=self.reparam_noise, max=1)
        
        return mu, sigma
    def sample_normal(self, state, reparameterize=True):
        mu, sigma = self.forward(state)
        probabilities = Normal(mu, sigma)
        
        if reparameterize:
            actions = probabilities.rsample()
        else:
            actions = probabilities.sample
            
        action = T.tanh(actions) * T.tensor(self.max_action).to(self.device)
        log_probs = probabilities.log_prob(actions)
        log_probs -= T.log(1 - action.pow(2) + self.reparam_noise)
        log_probs = log_probs.sum(1, keepdim=True)
        
        return action, log_probs
    
    def save_checkpoint(self):
        T.save(self.state_dict(), self.checkpoint_file)
        
    def load_checkpoint(self):
        self.load_state_dict_dict(T.load(self.checkpoint_file))
        

SyntaxError: cannot assign to function call (Temp/ipykernel_21064/2138916224.py, line 22)

In [10]:
#SAC agent
class Agent:
    def __init__(self, alpha = 0.0003, beta = 0.0003, input_dims=[8], env=None, gamma=0.99, n_actions=2, max_size=1000000, \
                 layer1_size=256, layer2_size=256, batch_size=256, reward_scale=2, tau=0.005):
        self.gamma = gamma
        self.tau = tau
        self.memory = ReplayBuffer(max_size, input_dims, n_actions)
        self.batch_size = batch_size
        self.n_actions = n_actions
        
        self.actor = ActorNetwork(alpha,input_dims, n_actions = n_actions, name = 'actor', max_action = env.action_space.high)
        self.critic1 = CriticNetwork(beta, input_dims, n_actions = n_actions, name='critic_1')
        self.critic2 = CriticNetwork(beta, input_dims, n_actions = n_actions, name='critic_2')
        self.value = ValueNetwork(beta, input_dims, name = 'value')
        self.target_value = ValueNetwork(beta, input_dims, name='target_value')
        
        self.scale = reward_scale
        self.update_network_parameters(tau=1)
        
    def choos_action(self, observation):
        state = T.tensor([observation]).to(self.actor.device)
        actions, _ = self.actor.sample_normal(state, reparameterize=False)
        
        return actions.cpu.detach().numpy()[0]
    
    def remember(self, state, action, reward, new_state, done):
        self.memory.store_transition(state, action, new_state, reward, done)
    
    def update_network_parameters(self, tau=None):
        if tau is None:
            tau = self.tau
        
        target_value_params = self.target_value.named_parameters()
        value_params = self.value.named_parameters()
        
        target_value_state_dict = dict(target_value_params)
        value_state_dict = dict(value_params)
        
        for name in value_state_dict:
            value_state_dict[name] = tau * value_state_dict[name].clone() + (1 - tau * target_value_state_dict[name].clone)
            
        self.target_value.load_state_dict(value_state_dict)
        
    def save_models(self):
        print('..... Saving model ....')
        self.actor.save_checkpoint()
        self.value.save_checkpoint()
        self.target_value.save_checkpoint()
        
        self.critic_1.save_checkpoint()
        self.critic_2.save_checkpoint()
        
        def load_models(self):
            print('..... Loading Models .....')
            self.value.load_checkpoint()
            self.target_value.load_checkpoint()
            self.critic_1.load_checkpoint()
            self.critic_2.load_checkpoint()
        
        def learn():
            if self.memory.mem_cntr < self.batch_size:
                return 
            
            state, action, reward, new_state, done = self.memory.sample_buffer(self.batch_size)
            
            reward = T.tensor(reward, dtype=T.float).to(self.actor.device)
            done = T.tensor(done).to(self.actor.device)
            state_ = T.tensor(new_state, dtype=T.float).to(self.actor.device)
            state = T.tensor(state, dtype=T.float).to(self.actor.device)
            action = T.tensor(action, dtype=T.float).to(self.actor.device)
            
            value = self.value(state).view(-1)
            value_ = self.target_value(state_).view(-1)
            value_[done] = 0.0
            
            actions, log_probs = self.actor.sample_normal(state, reparameterize=False)
            log_probs = log_probs.view(-1)
            q1_new_policy = self.critic_1.forward(state, actions)
            q2_new_policy = self.critic_2.forward(state, actions)
            critic_value = T.min(q1_new_policy, q2_new_policy)
            critic_value = critic_value.view(-1)
            
            self.value.optimizer.zero_grad()
            value_target = critic_value - log_probs
            value_los = 0.5 * F.mse_loss(value, value_target)
            value_loss.backward(retain_graph=True)
            self.value.optimizer.step()
            
            actions, log_probs = self.actor.sample_normal(state, reparameterize=True)
            log_probs = log_probs.view(-1)
            q1_new_policy = self.critic_1.forward(state, actions)
            q2_new_policy = self.critic_2.forward(state, actions)
            critic_value = T.min(q1_new_policy, q2_new_policy)
            critic_value = critic_value.view(-1)
            
            actor_loss = log_probs - critic_value
            actor_loss = T.mean(actor_loss)
            self.actor.optimizer.zero_grad()
            actor_loss.backward(retain_graph=True)
            self.actor.optimizer.step()
            
            self.critic_1.optimizer.zero_grad()
            self.critic_2.optimizer.zero_grad()
            q_hat = self.scale*reward + self.self.gamma*value_
            q1_old_policy = self.critic_1.forward(state, action).view(-1)
            q2_old_policy = self.critic_2.forward(state, action).view(-1)
            critic_1_loss = 0.5 * F.mse_loss(q1_old_policy, q_hat)
            critic_2_loss = 0.5 * F.mse_loss(q2_old_policy, q_hat)
            
            critic_loss = critic_1_loss + critic_2_loss
            critic_loss.backward()
            self.critic_1.optimizer.step()
            self.critic_2.optimizer.step()
            
            self.update_network_parameters()
        

In [11]:
import pybullet_envs
import gym
import numpy as np
from utils import plot_learning_curve

if __name__ == '__main__':
    env.gym.make('InvertedPendulumBulletEnv-v0')
    agent = Agent(input_dims=env.observation_space, env = env, n_actions = env.action_space.shape[0])
    n_games = 250
    file_name = 'inverted_pendulum.png'
    
    figure_file = 'plots/' + file_name
    
    best_score = env.reward_range[0]
    score_history = []
    load_checkpoint = False
    if load_checkpoint:
        agent.load_models()
        env.render(mode = 'human')
    
    for i in range(n_games):
        observation = env.reset()
        done = False
        score = 0 
        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.remember(observation, action, reward, observation_, done)
            if not load_checkpoint:
                agent.learn()
            observation = observation_
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])
        
        if avg_score > best_score:
            best_score = avg_score
            if not load_checkpoint:
                agent.save_models()
        
        print('episode ', i, 'score %.1f ' % score, 'avg_score %.1f' % avg_score)
    if not load_checkpoint:
        x = [i + 1 for i in range(n_games)]
        plot_learning_curve(x, score_history, figure_file)
            

ModuleNotFoundError: No module named 'utils'